# Seq-Seq model - one hot encoding - from keras tutorial 

## Data preparation and pre-processing

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Vectorize the data.

In [2]:
data_path= "./fra.txt"

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
lines[1:5]

['Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)']

We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.

In [3]:
num_samples = 10000  # Number of samples to train on.
test = []
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")

    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    test.append(input_text)

    #Make vectorization table by alphabets
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

### Sorting

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

#Calculate the max length of each sentence for padding
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


### Create token dictionaries

In [28]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

### One-Hot encoder and padding

3D nparray
[number of sentence, ,fixed sentence length, number of tokens]

In [6]:
# shape = (10000, 15, 71)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0 #Padding Encoder data
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


## Build Model

In [17]:
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens), name='encoder_inputs' ) #(None, Number of unique input tokens) = (None, None, 71)
encoder = keras.layers.LSTM(latent_dim, return_state=True, name= 'encoder') #encoding Spaces
encoder_outputs, state_h, state_c = encoder(encoder_inputs) #encoder_outputs.shape = (None, latent_dim), 

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens), name='decoder_inputs')

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) #decoder_outputs.shape = [None, max_length of output, 256]
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax", name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs) #Shape = [None, max_length of output, Number of unique output tokens] (None,59,92)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Decoder output is an one-hot encoding matrix

## Training

In [18]:
batch_size = 64
epochs = 20

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/20
125/125 [==============================] - 20s 148ms/step - loss: 1.1380 - accuracy: 0.7362 - val_loss: 1.0238 - val_accuracy: 0.7145
Epoch 2/20
125/125 [==============================] - 16s 128ms/step - loss: 0.8081 - accuracy: 0.7797 - val_loss: 0.8190 - val_accuracy: 0.7729
Epoch 3/20
125/125 [==============================] - 16s 126ms/step - loss: 0.6518 - accuracy: 0.8181 - val_loss: 0.6874 - val_accuracy: 0.8038
Epoch 4/20
125/125 [==============================] - 16s 129ms/step - loss: 0.5671 - accuracy: 0.8352 - val_loss: 0.6328 - val_accuracy: 0.8184
Epoch 5/20
125/125 [==============================] - 16s 129ms/step - loss: 0.5195 - accuracy: 0.8480 - val_loss: 0.5979 - val_accuracy: 0.8279
Epoch 6/20
125/125 [==============================] - 16s 128ms/step - loss: 0.4844 - accuracy: 0.8576 - val_loss: 0.5519 - val_accuracy: 0.8367
Epoch 7/20
125/125 [==============================] - 16s 130ms/step - loss: 0.4561 - accuracy: 0.8649 - val_loss: 0.5360 - val_ac

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


## Used trained model to test the translation
### Run inference (sampling)
1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [19]:
model = keras.models.load_model("s2s")
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 71)]   0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None, 92)]   0           []                               
                                                                                                  
 encoder (LSTM)                 [(None, 256),        335872      ['encoder_inputs[0][0]']         
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

### Reconstruct Encoder for states

In [20]:
# Restore the model and construct the encoder and decoder.
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

### Reconstruct Decoder

In [29]:
#Prepare for state input from previoius Encoder model
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#Reconstruct decoder model
decoder_inputs = model.input[1]  # input_2
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

### Making Reverse Dictionary

In [30]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

### Decoding

In [31]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #For the first alphabet
    # Generate empty target sequence of length 1 with all 0
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [39]:
item = np.random.choice(10000, 10)
for seq_index in item:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: I try.
Decoded sentence: J'ai contir.

-
Input sentence: I'm no saint.
Decoded sentence: Je ne suis pas seule.

-
Input sentence: Keep reading.
Decoded sentence: Continue à derlare.

-
Input sentence: Quit gambling.
Decoded sentence: Dis chante le soinnerais.

-
Input sentence: Taste it.
Decoded sentence: Goûtez-le.

-
Input sentence: Keep it.
Decoded sentence: Garde-le.

-
Input sentence: Are you happy?
Decoded sentence: Êtes-vous sûres ?

-
Input sentence: He's so cute.
Decoded sentence: Il est trop simé.

-
Input sentence: You can do it.
Decoded sentence: Vous avez l'air grasses.

-
Input sentence: They swam.
Decoded sentence: Ils sont la biens.

